<a href=#bookmark>**Once data is already scraped, click here to jump down to the right place.**</a>

Go to <a href=#bookmark2>my bookmark</a>

In [40]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline

import os
import pprint
import googlemaps
from collections import defaultdict
import time
import pickle

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import requests

chromedriver = f"/Users/brenner/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

%load_ext dotenv
%dotenv

%load_ext autoreload
%autoreload 2

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [316]:

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import requests

chromedriver = f"/Users/brenner/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [2]:
# Use fivethirtyeight styling, because I like it
import matplotlib.style as style
style.use('fivethirtyeight')

# Import private API key

MAPS_KEY = os.environ.get('MAPS_KEY')

# Change format of charts to .svg
%config InlineBackend.figure_format = 'svg'

In [4]:
%xmode

Exception reporting mode: Plain


In [5]:
# This dataset contains house sale prices for King County, which includes Seattle. It includes homes sold between May 2014 and May 2015.
# Read the data into pandas dataframe
df = pd.read_csv('kc_house_data.csv')

In [6]:
# Let's take a look at the data
df.head(3)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062


In [7]:
# Recast price column as integer, for simplicity
df['price'] = df.price.astype(int)

In [26]:
coords = list(zip(df['lat'].astype(float), df['long'].astype(float)))

In [30]:
len(coords)

21613

In [31]:
formatted_addresses = []
json_addresses = []
neighborhoods_list = []
address_dictionary = defaultdict(str)

def get_addresses(coords_list):
    count = 0
    for coord_tuple in coords_list:
        count +=1
#         if count >= 1000:
#             time.sleep(120)
#             count = 0
        
        try:
            # Instantiate a Google Maps API session
            address = gmaps.reverse_geocode(coord_tuple)
            
            # Parse the JSON results
            formatted_address = address[1]['formatted_address']
            
            # Store both a list of formatted addresses and a list of all data for safekeeping
            formatted_addresses.append(formatted_address)
            json_addresses.append(address)
            
            # Store a list of neighborhoods
            neighborhood = address[0]['address_components'][2]['short_name']
            neighborhoods_list.append(neighborhood)
            
            # Store a dictionary of lat/long and formatted addresses, in case we need to map to dataframe later
            address_dictionary[coord_tuple] = formatted_address
            
            print('Success')

        except:
            print('Could not get address.')
            formatted_addresses.append(None)

In [33]:
get_addresses(coords)

Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success


limit_output extension: Maximum message size of 10000 exceeded with 10016 characters

In [34]:
len(formatted_addresses)

21613

In [39]:
# # Pickling original list so not to lose my work
# with open('address_list_final.pkl', 'wb') as f:
#     pickle.dump(list_of_addresses, f)

# with open('formatted_addresses.pkl', 'wb') as f:
#     pickle.dump(formatted_addresses, f)

# # Pickling like crazy because I don't want to lose my data.
# # Yes, I know pickling isn't always the best. Works for now.
# with open('json_addresses.pkl', 'wb') as f:
#     pickle.dump(json_addresses, f)

# with open('neighborhoods.pkl', 'wb') as f:
#     pickle.dump(neighborhoods_list, f)

# with open('address_dictionary.pkl', 'wb') as f:
#     pickle.dump(address_dictionary, f)

In [64]:
# Backing up a whole bunch more
neighborhood_df.to_csv('neighborhood_csv.csv', index=False, header=False)
nh = pd.read_csv('neighborhood_csv.csv')
formatted_addresses_df = pd.DataFrame(formatted_addresses, columns=['Formatted Addresses'])
formatted_addresses_df.to_csv('formatted_addresses_csv.csv', index=False)



In [71]:
# Add formatted addresses from Google Maps API to our dataframe (!)
df['Address'] = formatted_addresses

In [ ]:
# Adding neighborhood to dataframe
df['Neighborhood'] = neighborhoods_list

**Start here once data is already scraped** <a name='bookmark' />

In [8]:
# Now that the data is already scraped, we can pick up from here and add 'Address' column to main dataframe
addresses_df = pd.read_csv('formatted_addresses_csv.csv')
df['Address'] = addresses_df['Formatted Addresses']

In [20]:
neighborhood_df = pd.read_csv('neighborhood_csv.csv', header=None)
df['Neighborhood'] = neighborhood_df

Code in cell below is for imputing zip code from Google addresses, but we already have that data in another column. May return to this later.

In [95]:
# # Grab a list of all the zip codes from the formatted addresses list
# # I love list comprehensions!
# zip_list = [x.split(',')[-2][-5:] for x in formatted_addresses]

# df['Imputed Zip Code'] = zip_list

# def get_zips(x):
#     if x == ' WA' or x == 'ngton':
#         return 0
#     else:
#         return x

# df['Imputed Zip Code'] = df['Imputed Zip Code'].apply(get_zips)

# df['Imputed Zip Code'] = df['Imputed Zip Code'].astype(int)

In [47]:
# 205 Unique Neighborhoods. Should be sufficient for Walk Score/Transit Score/Bike Score etc. for our purposes.
df['Neighborhood'].nunique()

205

In [48]:
# Initialize a numpy array with unique neighborhoods, that we will later use to scrape data from the web
unique_neighborhoods = np.array(df['Neighborhood'].value_counts().index)

In [54]:
df.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15', 'Address',
       'Neighborhood'],
      dtype='object')

In [165]:
adds = df['Address']

In [60]:
adds = adds.apply(lambda x: x.split(','))

In [68]:
adds = pd.DataFrame(adds)

In [160]:
adds

,Address
0,"[6101 S Cooper St, Seattle, WA 98118, USA]"
1,"[860 NE 127th St, Seattle, WA 98125, USA]"
2,"[15098 81st Ave NE, Kenmore, WA 98028, USA]"
3,"[9247 Fauntleroy Way SW, Seattle, WA 98136, ..."
4,"[757 222nd Pl NE, Sammamish, WA 98074, USA]"
5,"[25005 NE Patterson Way, Redmond, WA 98053, ..."
6,"[505 S 325th St, Federal Way, WA 98003, USA]"
7,"[21413 15th Ave S, Des Moines, WA 98198, USA]"
8,"[209 SW 102nd St, Seattle, WA 98146, USA]"
9,"[26019 232nd Pl SE, Maple Valley, WA 98038, ..."


In [163]:
# Parsing from the address column to get individual street numbers, cities, etc. to feed into Census
street_add = adds['Address'].apply(lambda x: x[0])

city = adds['Address'].apply(lambda x: x[1])

state = 'WA'

zip = adds['Address'].apply(lambda x: x[2][4:])

In [142]:
# Add a state column that doesn't have any wrong values
state = adds['Address'].apply(lambda x: x[2][1:3])

state = pd.DataFrame(state)

state['Address'] = 'WA'

state['Address'].value_counts()

WA    21613
Name: Address, dtype: int64

In [145]:
output = pd.concat([street_add, city, state, zip], axis=1, names=None)

In [289]:
output1 = output.iloc[8000:16000,:]

In [291]:
output1.head()

,Address,Address,Address,Address
8000,10261 39th Ave SW,Seattle,WA,98146
8001,17526 47th Ave NE,Lake Forest Park,WA,98155
8002,16116 NE 107th Ct,Redmond,WA,98052
8003,7024 126th Ave NE,Kirkland,WA,98033
8004,17242 164th Way SE,Renton,WA,98058


In [212]:
output1.to_csv('24000.csv')

In [172]:
new = pd.read_csv('output.csv')

In [362]:
census.head(3)

,index_num,Street,Match,Precision,Full address,loc,loc2,A,B,C,D,E
55,0,"6101 S Cooper St, Seattle, WA, 98118",Match,Non_Exact,"6101 S COOPER ST, SEATTLE, WA, 98118","-122.25696,47.51166",239766052.0,R,53.0,33.0,11900.0,NaN
53,1,"860 NE 127th St, Seattle, WA, 98125",Match,Non_Exact,"860 NE 127TH ST, SEATTLE, WA, 98125","-122.31958,47.721283",186716651.0,L,53.0,33.0,200.0,NaN
51,2,"15098 81st Ave NE, Kenmore, WA, 98028",Match,Exact,"15098 81ST AVE NE, KENMORE, WA, 98028","-122.23289,47.73783",239770253.0,R,53.0,33.0,22102.0,NaN


In [363]:
df.head(3)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,long,sqft_living15,sqft_lot15,Address,Neighborhood,walk_score,bike_score,transit_score,pers_crime_score,prop_crime_score
0,7129300520,20141013T000000,221900,3,1.00,1180,5650,1.0,0,0,...,-122.257,1340,5650,"6101 S Cooper St, Seattle, WA 98118, USA",Rainier Beach,NaN,NaN,NaN,NaN,NaN
1,6414100192,20141209T000000,538000,3,2.25,2570,7242,2.0,0,0,...,-122.319,1690,7639,"860 NE 127th St, Seattle, WA 98125, USA",North Seattle,NaN,NaN,NaN,NaN,NaN
2,5631500400,20150225T000000,180000,2,1.00,770,10000,1.0,0,0,...,-122.233,2720,8062,"15098 81st Ave NE, Kenmore, WA 98028, USA",Kenmore,NaN,NaN,NaN,NaN,NaN


In [364]:
df.tail(3)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,long,sqft_living15,sqft_lot15,Address,Neighborhood,walk_score,bike_score,transit_score,pers_crime_score,prop_crime_score
21610,1523300141,20140623T000000,402101,2,0.75,1020,1350,2.0,0,0,...,-122.299,1020,2007,"902 26th Ave S, Seattle, WA 98144, USA",Atlantic,NaN,NaN,NaN,NaN,NaN
21611,291310100,20150116T000000,400000,3,2.50,1600,2388,2.0,0,0,...,-122.069,1410,1287,"2157 NW Boulder Way Dr, Issaquah, WA 98027, USA",Talus,NaN,NaN,NaN,NaN,NaN
21612,1523300157,20141015T000000,325000,2,0.75,1020,1076,2.0,0,0,...,-122.299,1020,1357,"916 26th Ave S, Seattle, WA 98144, USA",Atlantic,NaN,NaN,NaN,NaN,NaN


In [365]:
census.tail(3)

,index_num,Street,Match,Precision,Full address,loc,loc2,A,B,C,D,E
3839,21610,"902 26th Ave S, Seattle, WA, 98144",Match,Exact,"902 26TH AVE S, SEATTLE, WA, 98144","-122.29906,47.594433",186593301.0,L,53.0,33.0,8900.0,NaN
3837,21611,"2157 NW Boulder Way Dr, Issaquah, WA, 98027",Match,Exact,"2157 NW BOULDER WAY DR, ISSAQUAH, WA, 98027","-122.06923,47.53435",187104382.0,L,53.0,33.0,25006.0,NaN
3845,21612,"916 26th Ave S, Seattle, WA, 98144",Match,Exact,"916 26TH AVE S, SEATTLE, WA, 98144","-122.29906,47.594334",186593301.0,L,53.0,33.0,8900.0,NaN


In [192]:
new = new[:10]

In [187]:
df3 = pd.read_csv('census_geocode_data/GeocodeResults - 1.csv')

In [189]:
df3.head(10)

,,,,,,,,,Unnamed: 0,"Address, Address, Address, Address",No_Match
6873,"6719 Silent Creek Ave SE, Snoqualmie, WA, 98065",Match,Exact,"6719 SILENT CREEK AVE SE, SNOQUALMIE, WA, 98065","-121.8784,47.540737",639757526.0,R,53.0,33.0,32602.0,NaN
6874,"14718 NE 16th St, Bellevue, WA, 98007",Tie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6871,"26012 18th Ave S, Des Moines, WA, 98198",Match,Exact,"26012 18TH AVE S, DES MOINES, WA, 98198","-122.31009,47.36838",619973594.0,R,53.0,33.0,30003.0,NaN
6872,"9735 NE 138th Pl, Kirkland, WA, 98034",Match,Non_Exact,"9735 NE 138TH PL, KIRKLAND, WA, 98034","-122.21162,47.724415",239780830.0,L,53.0,33.0,22202.0,NaN
6870,"15210 Bridle Crest Trail, Redmond, WA, 98052",No_Match,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
531,"2904 28th St SE, Auburn, WA, 98092",Match,Exact,"2904 28TH ST SE, AUBURN, WA, 98092","-122.190445,47.28541",186656868.0,R,53.0,33.0,31100.0,NaN
530,"21315 SE 269th St, Maple Valley, WA, 98038",Match,Non_Exact,"21315 SE 269TH ST, MAPLE VALLEY, WA, 98038","-122.05633,47.360542",239781411.0,R,53.0,33.0,32008.0,NaN
2951,"15436 SE 20th Pl, Bellevue, WA, 98007",Match,Non_Exact,"15436 SE 20TH PL, BELLEVUE, WA, 98007","-122.13292,47.592213",186614363.0,L,53.0,33.0,23401.0,NaN
6869,"27369 245th Ave SE, Maple Valley, WA, 98038",Match,Exact,"27369 245TH AVE SE, MAPLE VALLEY, WA, 98038","-122.01567,47.356506",186958433.0,R,53.0,33.0,31603.0,NaN
2950,"1819 N 145th St, Shoreline, WA, 98133",Match,Non_Exact,"1819 N 145TH ST, SHORELINE, WA, 98133","-122.33673,47.734127",186574963.0,R,53.0,33.0,300.0,NaN


In [222]:
census_1 = pd.read_csv('census_geocode_data/0 - 8000.csv', header=None)

In [225]:
census_2 = pd.read_csv('census_geocode_data/8-16000.csv', header=None)

In [258]:
census_3 = pd.read_csv('census_geocode_data/24000_csv.csv', header=None)

In [280]:
census = pd.concat([census_1, census_2, census_3], ignore_index=False)

In [295]:
names=['index_num', 'Street', 'Match', 'Precision', 'Full address', 'loc', 'loc2', 'A', 'B', 'C', 'D', 'E']

In [296]:
census.columns = names

In [261]:
len(census_1)

8000

In [265]:
census_3.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11
0,16902,"14444 129th Ave NE, Kirkland, WA, 98034",Match,Non_Exact,"14444 129TH PL NE, KIRKLAND, WA, 98034","-122.16729,47.73239",186600250.0,R,53.0,33.0,21906.0,NaN
1,20439,"803 SW 4th Pl, Renton, WA, 98057",No_Match,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [303]:
census.sort_values('index_num', inplace=True)

In [304]:
census.tail()

,index_num,Street,Match,Precision,Full address,loc,loc2,A,B,C,D,E
3862,21608,"929 N 97th St, Seattle, WA, 98103",Match,Non_Exact,"929 N 97TH ST, SEATTLE, WA, 98103","-122.34681,47.699615",239761412.0,R,53.0,33.0,1300.0,NaN
3832,21609,"2245 SW 104th St, Seattle, WA, 98146",Match,Non_Exact,"2245 SW 104TH ST, SEATTLE, WA, 98146","-122.36205,47.51002",186623179.0,L,53.0,33.0,26600.0,NaN
3839,21610,"902 26th Ave S, Seattle, WA, 98144",Match,Exact,"902 26TH AVE S, SEATTLE, WA, 98144","-122.29906,47.594433",186593301.0,L,53.0,33.0,8900.0,NaN
3837,21611,"2157 NW Boulder Way Dr, Issaquah, WA, 98027",Match,Exact,"2157 NW BOULDER WAY DR, ISSAQUAH, WA, 98027","-122.06923,47.53435",187104382.0,L,53.0,33.0,25006.0,NaN
3845,21612,"916 26th Ave S, Seattle, WA, 98144",Match,Exact,"916 26TH AVE S, SEATTLE, WA, 98144","-122.29906,47.594334",186593301.0,L,53.0,33.0,8900.0,NaN


In [308]:
# for col in ['B', 'C', 'D']:
#     census[col] = census[col].astype('int')

ValueError: Cannot convert non-finite values (NA or inf) to integer

In [305]:
df.tail()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,Address,Neighborhood
21608,263000018,20140521T000000,360000,3,2.50,1530,1131,3.0,0,0,...,0,2009,0,98103,47.6993,-122.346,1530,1509,"929 N 97th St, Seattle, WA 98103, USA",Greenwood
21609,6600060120,20150223T000000,400000,4,2.50,2310,5813,2.0,0,0,...,0,2014,0,98146,47.5107,-122.362,1830,7200,"2245 SW 104th St, Seattle, WA 98146, USA",Highline
21610,1523300141,20140623T000000,402101,2,0.75,1020,1350,2.0,0,0,...,0,2009,0,98144,47.5944,-122.299,1020,2007,"902 26th Ave S, Seattle, WA 98144, USA",Atlantic
21611,291310100,20150116T000000,400000,3,2.50,1600,2388,2.0,0,0,...,0,2004,0,98027,47.5345,-122.069,1410,1287,"2157 NW Boulder Way Dr, Issaquah, WA 98027, USA",Talus
21612,1523300157,20141015T000000,325000,2,0.75,1020,1076,2.0,0,0,...,0,2008,0,98144,47.5941,-122.299,1020,1357,"916 26th Ave S, Seattle, WA 98144, USA",Atlantic


In [53]:
df.iloc[0, df.columns.get_loc('walk_score')] = 4

In [54]:
df.head(2)

,Unnamed: 0,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,...,sqft_lot15,Address,Neighborhood,walk_score,bike_score,transit_score,pers_crime_score,prop_crime_score,GEOID,income
0,0,7129300520,20141013T000000,221900,3,1.00,1180,5650,1.0,0,...,5650,"6101 S Cooper St, Seattle, WA 98118, USA",Rainier Beach,4.0,NaN,NaN,NaN,NaN,53033011900,71524.0
1,1,6414100192,20141209T000000,538000,3,2.25,2570,7242,2.0,0,...,7639,"860 NE 127th St, Seattle, WA 98125, USA",North Seattle,NaN,NaN,NaN,NaN,NaN,53033000200,56208.0


In [314]:
df.to_csv('dataframe 10.05.18 4 PM')

16000.csv                    TEST.csv
24000.csv                    Walk Score Scraping.ipynb
8000.csv                     census_geocode_data
Housing Data.ipynb           dataframe 10.05.18 4 PM
Luther.jpg                   formatted_addresses_csv.csv
Project Luther - Proposal.md kc_house_data.csv
README.md                    neighborhood_csv.csv


In [23]:
# # Create dicts for Walk, Transit, & Bike Scores
# walk_scores = defaultdict(str)
# transit_scores = defaultdict(str)
# bike_scores = defaultdict(str)

# # Create dicts for Personal & Property Crime Scores
# personal_crime_scores = defaultdict(str)
# property_crime_scores = defaultdict(str)

In [56]:
address = '1436 19th Ave., Seattle WA'
url_address = address.lower().replace(",", "").replace('.', '').replace(" ", "-")
print(new)

1436-19th-ave-seattle-wa


In [95]:
# Must have started a Webdriver/Selenium session, e.g.:


def get_walk_scores(start, num_records_to_fetch):

    count = 0
    error_count = 0
    sleep_times = []
    
    for i in range(start, start + num_records_to_fetch + 1):
        address = df.iloc[i, df.columns.get_loc('Address')]
        print(f'Starting at index {i}.')
        
        if count==0:
            # Fetch the URL
            driver.get(f'https://www.walkscore.com/score/{new}')
            url_address = address.lower().replace(",", "").replace('.', '').replace(" ", "-")
            driver.get(f'https://www.walkscore.com/score/{url_address}')
        
        # Use search bar to get our next page
        
        try:
            input_element = driver.find_element_by_id('addrbar-street')
            input_element.click()
            input_element.send_keys(address)
            input_element.send_keys(Keys.ENTER)
            
        except:
            error_count += 1
            print('Search bar didn\'t work. Entering address manually.')
            url_address = address.lower().replace(",", "").replace('.', '').replace(" ", "-")
            driver.get(f'https://www.walkscore.com/score/{url_address}')
        
        # Sleep a bit
        sleep_time = abs(1 + (np.random.randn(1)))
        time.sleep(sleep_time)
        sleep_times.append(sleep_time)
        print(f'Sleeping for {sleep_time} seconds.')
        
        # Read the html
        html = driver.page_source
        soup=BeautifulSoup(html)

        # Getting Walk, Transit, and Bike Scores
        image_tags = soup.find_all('img')
        
        try:
            for score in image_tags:
                if "Score of" in str(score):
                    
                    z = score['src']
                    
                    #Figure out which score (Walk/Transit/Bike) this is
                    if 'walk/' in z:
                        walk_scores[address] = z[-6:-4]
                        df.iloc[i, df.columns.get_loc('walk_score')] = z[-6:-4]
                    if 'transit/' in z:
                        transit_scores[address] = z[-6:-4]
                        df.iloc[i, df.columns.get_loc('transit_score')] = z[-6:-4]
                    if 'bike/' in z:
                        bike_scores[address] = z[-6:-4]
                        df.iloc[i, df.columns.get_loc('bike_score')] = z[-6:-4]

        except:
            print('No Walk/Bike/Transit Score for {address}')

        # Getting Personal & Property Crime Grades
        results = soup.find_all("div", {"class" : "crime-grade"})
        parsed_grades = []
            
        try:
            for grade in results:
                parsed_grades.append(grade.text)

            personal_crime_scores[address] = parsed_grades[0][2]
            property_crime_scores[address] = parsed_grades[1][2]
            
            df.iloc[i, df.columns.get_loc('pers_crime_score')] = parsed_grades[0][2]
            df.iloc[i, df.columns.get_loc('prop_crime_score')] = parsed_grades[1][2]
            
        except:
            pass
        
        print(f'{count} pages scraped. On index {i}.')
        
        count +=1
        
        if count % 10 == 0:
            print(f'Average sleep time between scrapes: {np.mean(sleep_times)} seconds.')
        
        if count % 50 == 0:
            df.to_csv('backup_df_from_scraping.csv')
            time.sleep(30)
            continue
        
        # Move on to the next page

            


In [96]:
driver = webdriver.Chrome(chromedriver)
driver.get(f'https://www.walkscore.com/score/1430-18th-ave-seattle-wa-98122')
most_recent_record = df[df['walk_score'].isnull() == True].index[0]
get_walk_scores(most_recent_record, 300)

Starting at index 32.
Sleeping for [3.47220928] seconds.
0 pages scraped. On index 32.
Starting at index 33.
Sleeping for [5.47273409] seconds.
1 pages scraped. On index 33.
Starting at index 34.
Sleeping for [3.38885334] seconds.
2 pages scraped. On index 34.
Starting at index 35.
Sleeping for [1.71590787] seconds.
3 pages scraped. On index 35.
Starting at index 36.
Sleeping for [4.25247578] seconds.
4 pages scraped. On index 36.
Starting at index 37.
Sleeping for [3.14761406] seconds.
5 pages scraped. On index 37.
Starting at index 38.
Sleeping for [1.47049591] seconds.
6 pages scraped. On index 38.
Starting at index 39.
Sleeping for [5.2380819] seconds.
7 pages scraped. On index 39.
Starting at index 40.
Sleeping for [3.0173124] seconds.
8 pages scraped. On index 40.
Starting at index 41.
Sleeping for [2.76447227] seconds.
9 pages scraped. On index 41.
Starting at index 42.
Sleeping for [4.50198816] seconds.
10 pages scraped. On index 42.
Starting at index 43.
Sleeping for [4.474530

In [97]:
df[200:250]

,Unnamed: 0,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,...,sqft_lot15,Address,Neighborhood,walk_score,bike_score,transit_score,pers_crime_score,prop_crime_score,GEOID,income
200,200,3542300060,20150311T000000,210000,3,1.00,860,11725,1.0,0,...,9514,"204 Glennwood Ave NE, Renton, WA 98056, USA",Renton,27,NaN,33,NaN,NaN,53033025601,60833.0
201,201,2222059065,20141112T000000,297000,3,2.50,1940,14952,2.0,0,...,10450,"24933 132nd Pl SE, Kent, WA 98042, USA",Kent,33,NaN,26,NaN,NaN,53033031703,84408.0
202,202,7551300060,20140716T000000,470000,3,1.00,1010,5000,1.0,0,...,5000,"2839 NW 64th St, Seattle, WA 98107, USA",Old Ballard,72,77,47,A,B,53033003200,79681.0
203,203,100600550,20140804T000000,226500,3,1.50,1300,7370,1.0,0,...,7500,"33400 28th Pl SW, Federal Way, WA 98023, USA",Federal Way,50,NaN,35,NaN,NaN,53033030311,71659.0
204,204,3211100860,20150303T000000,274250,3,1.00,910,8450,1.0,0,...,8040,"4512 SE 4th Pl, Renton, WA 98059, USA",Renton,26,NaN,26,NaN,NaN,53033000000,NaN
205,205,3456000310,20140804T000000,840000,4,1.75,2480,11010,1.0,0,...,10744,"8866 SE 72nd Pl, Mercer Island, WA 98040, USA",Mercer Island,31,NaN,23,NaN,NaN,53033024602,195156.0
206,206,9526600140,20140919T000000,677900,3,2.50,2440,4587,2.0,0,...,4587,"16925 NE 120th St, Redmond, WA 98052, USA",North Redmond,/3,NaN,26,NaN,NaN,53033032321,199542.0
207,207,7465900060,20150205T000000,425000,3,1.00,1010,5864,1.0,0,...,5000,"3275 38th Ave SW, Seattle, WA 98126, USA",North Admiral,65,57,50,A,A,53033009800,71705.0
208,208,1222000055,20141123T000000,180250,2,0.75,900,9600,1.0,0,...,14280,"4th Ave SW & SW 160th St, Burien, WA 98166, USA",Burien,83,NaN,49,NaN,NaN,53033000000,NaN
209,209,6300000550,20140717T000000,464000,6,3.00,2300,3404,2.0,0,...,1312,"10549 Stone Ave N, Seattle, WA 98133, USA",Haller Lake,81,66,59,D,C,53033001300,56744.0


In [90]:
df[25:60]

,Unnamed: 0,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,...,sqft_lot15,Address,Neighborhood,walk_score,bike_score,transit_score,pers_crime_score,prop_crime_score,GEOID,income
25,25,1202000200,20141103T000000,233000,3,2.00,1710,4697,1.5,0,...,4705,"227 I St SE, Auburn, WA 98002, USA",Auburn,75,NaN,38,NaN,NaN,53033000000,NaN
26,26,1794500383,20140626T000000,937000,3,1.75,2450,2691,2.0,0,...,3573,"2215 2nd Ave W, Seattle, WA 98119, USA",West Queen Anne,94,60,63,A,B,53033006800,98750.0
27,27,3303700376,20141201T000000,667000,3,1.00,1400,1581,1.5,0,...,3861,"1414 E Harrison St, Seattle, WA 98112, USA",Capitol Hill,95,69,77,C,C,53033007500,63419.0
28,28,5101402488,20140624T000000,438000,3,1.75,1520,6380,1.0,0,...,6235,"9109 23rd Ave NE, Seattle, WA 98115, USA",Northeast Seattle,71,59,51,A,B,53033002100,85759.0
29,29,1873100390,20150302T000000,719000,4,2.50,2570,7173,2.0,0,...,6026,"17260 NE 119th Way, Redmond, WA 98052, USA",North Redmond,/8,NaN,27,NaN,NaN,53033032321,199542.0
30,30,8562750320,20141110T000000,580500,3,2.50,2320,3980,2.0,0,...,3980,"2190 NW Coyote Creek Ln, Issaquah, WA 98027, USA",Talus,/9,NaN,41,NaN,NaN,53033025006,148750.0
31,31,2426039314,20141201T000000,280000,2,1.50,1190,1265,3.0,0,...,1756,"13437 Greenwood Ave N, Seattle, WA 98133, USA",Broadview,53,57,49,B,B,53033000401,32472.0
32,32,461000390,20140624T000000,687500,4,1.75,2330,5000,1.5,0,...,5000,"7318 9th Ave NW, Seattle, WA 98117, USA",Ballard,NaN,NaN,NaN,NaN,NaN,53033003000,90789.0
33,33,7589200193,20141110T000000,535000,3,1.00,1090,3000,1.5,0,...,5080,"1414 NW 83rd St, Seattle, WA 98117, USA",Ballard,NaN,NaN,NaN,NaN,NaN,53033003000,90789.0
34,34,7955080270,20141203T000000,322500,4,2.75,2060,6659,1.0,0,...,8720,"19416 138th Ave SE, Renton, WA 98058, USA",Renton,NaN,NaN,NaN,NaN,NaN,53033029307,94375.0


In [47]:
alt_df = pd.read_csv('backup_df_from_scraping.csv')

In [330]:
# # Created new columns to hold scores
# df['walk_score'] = np.nan
# df['bike_score'] = np.nan
# df['transit_score'] = np.nan
# df['pers_crime_score'] = np.nan
# df['prop_crime_score'] = np.nan

**Adding GEOID and income data to dataframe <a name='bookmark2' />**

In [393]:
# Fill NAs with float of 0.0 so we can run operations on the whole column to transform into full GEOID
census['D'].fillna(0.0, inplace=True)

In [398]:
# Add leading zeros to transform Census Tract ID into format we can use to find GEOID to match up with Census data
def add_zeros(x):
    x = str(int(x))
    x = x.zfill(6)
    return(x)

census['tract'] = census['D'].apply(add_zeros)

In [407]:
# Add the leading characters for GEOID to our tract number
# Note: '53' denotes Washington State, and '033' denotes King County.
census['GEOID'] = '53033' + census['tract']

In [411]:
# Add 'GEOID' column to our main dataframe
census.df['GEOID'] = census['GEOID'].values

,index_num,Street,Match,Precision,Full address,loc,loc2,A,B,C,D,E,tract,GEOID
55,0,"6101 S Cooper St, Seattle, WA, 98118",Match,Non_Exact,"6101 S COOPER ST, SEATTLE, WA, 98118","-122.25696,47.51166",239766052.0,R,53.0,33.0,11900.0,NaN,011900,53033011900
53,1,"860 NE 127th St, Seattle, WA, 98125",Match,Non_Exact,"860 NE 127TH ST, SEATTLE, WA, 98125","-122.31958,47.721283",186716651.0,L,53.0,33.0,200.0,NaN,000200,53033000200
51,2,"15098 81st Ave NE, Kenmore, WA, 98028",Match,Exact,"15098 81ST AVE NE, KENMORE, WA, 98028","-122.23289,47.73783",239770253.0,R,53.0,33.0,22102.0,NaN,022102,53033022102
49,3,"9247 Fauntleroy Way SW, Seattle, WA, 98136",Match,Exact,"9247 FAUNTLEROY WAY SW, SEATTLE, WA, 98136","-122.3935,47.520912",186661942.0,R,53.0,33.0,11600.0,NaN,011600,53033011600
73,4,"757 222nd Pl NE, Sammamish, WA, 98074",Match,Exact,"757 222ND PL NE, SAMMAMISH, WA, 98074","-122.0446,47.61503",187001283.0,L,53.0,33.0,32317.0,NaN,032317,53033032317


In [590]:
median_income = pd.read_csv('median_income.csv')

In [592]:
income_dict = defaultdict(str)

In [438]:
# Remove the last entry because it contains a null value
median_income = median_income.iloc[:-1, :]

In [479]:
# Found another incorrect value. Finding the index so I can drop it.
median_income[median_income['HC02_EST_VC02'] == '(X)']

,GEO.id2,HC02_EST_VC02
54,53033005302,(X)


In [484]:
median_income.iloc[54]

GEO.id2          53033005302
HC02_EST_VC02            (X)
Name: 54, dtype: object

In [488]:
# Dropping incorrect row.
median_income = median_income.drop(54)

In [583]:
# Reset index in order to make zipping into dicts possible without errors
median_income = median_income.reset_index(inplace=True)

In [556]:
# Recasting income variable as an int to make things easier down the road
median_income['HC02_EST_VC02'] = median_income['HC02_EST_VC02'].astype('str', inplace=True)

In [582]:
# Backing up work
df.to_csv('backup saturday.csv')
census.to_csv('census backup saturday.csv')
median_income.to_csv('median_income backup.csv')

In [7]:
df = pd.read_csv('backup saturday.csv')

census = pd.read_csv('census backup saturday.csv')

median_income = pd.read_csv('median_income backup.csv')

In [14]:
median_income.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396 entries, 0 to 395
Data columns (total 4 columns):
Unnamed: 0       396 non-null int64
index            396 non-null int64
GEO.id2          396 non-null int64
HC02_EST_VC02    396 non-null int64
dtypes: int64(4)
memory usage: 12.5 KB


In [9]:
median_income['GEO.id2']

0      53033000100
1      53033000200
2      53033000300
3      53033000401
4      53033000402
5      53033000500
6      53033000600
7      53033000700
8      53033000800
9      53033000900
10     53033001000
11     53033001100
12     53033001200
13     53033001300
14     53033001400
15     53033001500
16     53033001600
17     53033001701
18     53033001702
19     53033001800
20     53033001900
21     53033002000
22     53033002100
23     53033002200
24     53033002400
25     53033002500
26     53033002600
27     53033002700
28     53033002800
29     53033002900
          ...     
366    53033032214
367    53033032215
368    53033032307
369    53033032309
370    53033032311
371    53033032313
372    53033032315
373    53033032316
374    53033032317
375    53033032318
376    53033032319
377    53033032320
378    53033032321
379    53033032322
380    53033032323
381    53033032324
382    53033032325
383    53033032326
384    53033032327
385    53033032328
386    53033032329
387    53033

In [15]:
income_tracts = dict(zip(median_income['GEO.id2'], median_income['HC02_EST_VC02']))

In [16]:
income_tracts

{53033000100: 44951,
 53033000200: 56208,
 53033000300: 64297,
 53033000401: 32472,
 53033000402: 62726,
 53033000500: 125400,
 53033000600: 60131,
 53033000700: 51741,
 53033000800: 87222,
 53033000900: 125109,
 53033001000: 69643,
 53033001100: 64549,
 53033001200: 31863,
 53033001300: 56744,
 53033001400: 72428,
 53033001500: 95927,
 53033001600: 96193,
 53033001701: 65188,
 53033001702: 62786,
 53033001800: 87159,
 53033001900: 61391,
 53033002000: 80400,
 53033002100: 85759,
 53033002200: 132981,
 53033002400: 103917,
 53033002500: 101413,
 53033002600: 91458,
 53033002700: 111661,
 53033002800: 90568,
 53033002900: 103750,
 53033003000: 90789,
 53033003100: 101397,
 53033003200: 79681,
 53033003300: 85595,
 53033003400: 97500,
 53033003500: 81563,
 53033003600: 65172,
 53033003800: 96310,
 53033003900: 110458,
 53033004000: 75714,
 53033004100: 132917,
 53033004200: 110091,
 53033004301: 64797,
 53033004302: 18908,
 53033004400: 50500,
 53033004500: 88914,
 53033004600: 113906,
 

In [17]:
df.head()

,Unnamed: 0,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,...,sqft_living15,sqft_lot15,Address,Neighborhood,walk_score,bike_score,transit_score,pers_crime_score,prop_crime_score,GEOID
0,0,7129300520,20141013T000000,221900,3,1.00,1180,5650,1.0,0,...,1340,5650,"6101 S Cooper St, Seattle, WA 98118, USA",Rainier Beach,NaN,NaN,NaN,NaN,NaN,53033011900
1,1,6414100192,20141209T000000,538000,3,2.25,2570,7242,2.0,0,...,1690,7639,"860 NE 127th St, Seattle, WA 98125, USA",North Seattle,NaN,NaN,NaN,NaN,NaN,53033000200
2,2,5631500400,20150225T000000,180000,2,1.00,770,10000,1.0,0,...,2720,8062,"15098 81st Ave NE, Kenmore, WA 98028, USA",Kenmore,NaN,NaN,NaN,NaN,NaN,53033022102
3,3,2487200875,20141209T000000,604000,4,3.00,1960,5000,1.0,0,...,1360,5000,"9247 Fauntleroy Way SW, Seattle, WA 98136, USA",Fauntleroy,NaN,NaN,NaN,NaN,NaN,53033011600
4,4,1954400510,20150218T000000,510000,3,2.00,1680,8080,1.0,0,...,1800,7503,"757 222nd Pl NE, Sammamish, WA 98074, USA",Sammamish,NaN,NaN,NaN,NaN,NaN,53033032317


In [19]:
df['income'] = df['GEOID'].map(income_tracts)

In [21]:
income_tracts[53033022102]

96863